# CNN Model

This Convolutional Neural Network will identify features in an image and will pass these features onto the XGBoost model to determine the nature of the skin lesion.

## Step 1: Import Libraries

We will be importing the necessary libraries needed in order to develop this CNN.

In [14]:
# Import Necessary Libraries
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import kagglehub

## Step 2: Initializing Data

Process the data so that the model will be able to understand it. We will be using Kaggle's Skin Cancer Database.

In [22]:
# Step 2.1: Load Dataset
dataset_path = kagglehub.dataset_download("farjanakabirsamanta/skin-cancer-dataset") # Get Data from Kaggle Database
print("Data set path: ", dataset_path + "/HAM10000_metadata.csv")

df = pd.read_csv(dataset_path + "/HAM10000_metadata.csv")
df.head() # See data in csv form before processing

Data set path:  C:\Users\leyan\.cache\kagglehub\datasets\farjanakabirsamanta\skin-cancer-dataset\versions\1/HAM10000_metadata.csv


,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [ ]:
# Step 2.2: Processing Data
# We will be using Pillow to load in the data as an array
from PIL import Image

# Receive Notification after data is fully loaded into this array :)
from notifypy import Notify

# CAN CHANGE:
NUMBER_IMAGES = 100

notification = Notify()
notification.title = "Started Loading Data"
notification.message = "Currently loading: " + str(NUMBER_IMAGES) + " Images"
notification.send()

images = np.array([np.asarray(Image.open(dataset_path + "/Skin Cancer/Skin Cancer/" + id + ".jpg")) for id in df.loc[0:NUMBER_IMAGES-1, "image_id"]])
print(images[0][100, 100])

notification = Notify()
notification.title = "Data Finished Loading"
notification.message = "Successfully loaded: " + str(NUMBER_IMAGES) + " Images"
notification.send()

# Create Labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
labels = np.array([
    
])
labels.head()


[215 170 199]


AttributeError: 'numpy.ndarray' object has no attribute 'head'

## Step 3: Train Model

In this step, we will create the CNN models and train them to identify structures in an image.

In [68]:
# Main Model
model = Sequential([
    Conv2D(32, (3, 3), activation="relu", input_shape=(450, 600, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

"""
model = Sequential()

model.add(Conv2D(32, (3, 3), activation="relu", input_shape=(450, 600, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(10, activation="sigmoid"))
"""

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(images.shape)
model.fit(images, labels, epochs=3)

(100, 450, 600, 3)


ValueError: Invalid dtype: object